Feedback to shins777@gmail.com

이 예제는 구글의 BigQuery를 Vector database로 활용하는 예제입니다.
현재 상용화되거나, 오픈소스의 다양한 Vector DB가 있지만, BigQuery의 다양한 기능과, VectorDB 의 특화된 기능이 검색 성능을 높히고 효율적인 개발환경을 구성할 수 있습니다.
여기서 사용하는 임베딩 모델은 사용자가 준비한 embedding 모듈을 사용하도록 구성하였습니다.

이 예제는 Langchain API 기준으로 설명합니다.



# 라이브러리 설치

In [ ]:
!pip install --upgrade --quiet sentence_transformers langchain langchain-google-vertexai google-cloud-aiplatform google-cloud-bigquery

# GCP 인증 및 환경설정

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
PROJECT_ID="PROJECT_ID"
REGION="asia-northeast3"
MODEL = "gemini-pro"

#set and show gcp project
!gcloud config set project {PROJECT_ID}
!gcloud config get-value project

# Custom Embedding 설정

Custom embedding 사용시 Langchain 에 있는 함수를 활용려면 Ebeddings interface내에 두개의 함수를 구현해야 합니다.

*   Ebeddings interface 구현 : https://api.python.langchain.com/en/stable/embeddings/langchain_core.embeddings.Embeddings.html#langchain_core.embeddings.Embeddings
*   아래 두 함수 구현
  *   abstract embed_documents(texts: List[str]) → List[List[float]]
  *   embed_query(text: str) → List[float]





In [ ]:
from langchain_core.embeddings import Embeddings
from typing import List

from sentence_transformers import SentenceTransformer

class Custom_Embedding(Embeddings):

  model = None

  def __init__(self, model_name: str):
    self.model = SentenceTransformer(model_name)

  def embed_documents(self, texts: List[str]) -> List[List[float]]:
    embeddings = self.model.encode(texts)
    return embeddings.tolist()

  def embed_query(self, text: str) -> List[float]:
    embeddings = self.model.encode([text])
    return embeddings.tolist()[0]


# Custom Embeddings

Hugging face에서 비교적 검증된 임베딩 모듈을 사용합니다. 사용자가 원하는 임베딩 모듈을 구성해주세요.

*   https://huggingface.co/snunlp/KR-SBERT-V40K-klueNLI-augSTS
*   https://huggingface.co/sentence-transformers/stsb-xlm-r-multilingual




In [ ]:
# EBEDDING_MODEL = "sentence-transformers/stsb-xlm-r-multilingual"
EBEDDING_MODEL = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
embedding = Custom_Embedding(EBEDDING_MODEL)


In [ ]:
from google.cloud import bigquery

DATASET = "vector_db_custom"
TABLE = "vector_table_custom"

client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)



*   https://python.langchain.com/docs/integrations/vectorstores/bigquery_vector_search
*   https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch



In [ ]:
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

table = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,

    #https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.utils.DistanceStrategy.html#langchain_community.vectorstores.utils.DistanceStrategy
    distance_strategy=DistanceStrategy.COSINE

)

In [ ]:
import pandas as pd

terms = pd.read_csv('./term1.csv',sep="|", encoding='utf-8-sig')
terms

In [ ]:
import json

all_texts = terms['context'].to_list()
#metadatas = [ {'context_title': row['context_title'] } for idx, row in terms.iterrows()]
#table.add_texts(all_texts_list, metadatas=metadatas)
table.add_texts(all_texts)


# Sentence similarity 데이터 조회


*   https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.similarity_search



In [ ]:
import time
s = time.time()

query = "질문을 넣어주세요."

docs = table.similarity_search(query, k=5)

for doc in docs:
  print(doc.page_content)

e = time.time() - s
print(e)

# Vector 형태의 쿼리 조회

In [ ]:
query_vector = embedding.embed_query(query)
docs = table.similarity_search_by_vector(query_vector, k=5)
for doc in docs:
  print(doc.page_content)

# 쿼리 유사성 다양성 최적화

*   https://api.python.langchain.com/en/stable/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html#langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.max_marginal_relevance_search

In [ ]:
docs = table.max_marginal_relevance_search(query= query,
                                           k=5,
                                           fetch_k = 30,
                                           lambda_mult = 0.5,
                                           brute_force = True
                                           )
for doc in docs:
  print(doc.page_content)

# Similarity with Score

In [ ]:
tuples = table.similarity_search_with_relevance_scores(query, k=5)

context ={}

for tp in tuples:
    context[tp[1]] = tp[0].page_content
    # print(f"==[{tp[1]}]==")
    # print(tp[0].page_content)

context

# Gemini Pro 실행 - BigQuery as a Grounding Service

Responsible AI setting
*   HarmCategory : https://cloud.google.com/vertex-ai/docs/reference/rest/v1/HarmCategory
*   HarmBlockThreshold : https://cloud.google.com/php/docs/reference/cloud-ai-platform/0.31.0/V1.SafetySetting.HarmBlockThreshold

In [ ]:
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory

safety_settings = {
                    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE
}

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [ ]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  safety_settings = safety_settings,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

query = "질문을 넣어주세요."

prompt = PromptTemplate.from_template("""

  당신은 법률을 상담하는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=query)

print(f"Prompt : {prompt}")
print(f"답변 : {gemini_pro.invoke(prompt)}")
